In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://graph.node.bean.money/subgraphs/name/beanstalk


In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [3]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season', 'timestamp', 'harvestablePods', 'harvestedPods', 'unharvestablePods']) 

In [8]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns={
    'harvestablePods': 'harvestable_pods', 
    'harvestedPods': 'harvested_pods', 
    'unharvestablePods': 'unharvestable_pods', 
})
df_field['unharvestable_ratio'] = df_field.unharvestable_pods / (
    df_field.harvested_pods + df_field.unharvestable_pods + df_field.harvestable_pods
)
df_field.head()

,season,harvestable_pods,harvested_pods,timestamp,unharvestable_pods,unharvestable_ratio
0,25,5049.266341,0.000000,2021-08-07 00:06:08,4899.059312,0.492451
1,49,337.098080,15526.430505,2021-08-08 00:08:15,18029.272057,0.531950
2,73,1479.961469,21827.924952,2021-08-09 00:15:57,28649.320656,0.551402
3,97,7611.092329,25843.746295,2021-08-10 00:01:29,25843.746295,0.435824
4,121,6904.487041,26550.351583,2021-08-11 00:00:05,26550.351583,0.442468


In [9]:
id_vars = ['timestamp', 'season']
value_vars = [
   'unharvestable_pods', 'harvested_pods', 'harvestable_pods', 
   'unharvestable_ratio'
]
df = wide_to_longwide(df_field, "timestamp", id_vars, value_vars, sort_col="timestamp") 
df.head()

,timestamp,season,variable,value,unharvestable_pods,harvested_pods,harvestable_pods,unharvestable_ratio
0,2021-08-07 00:06:08,25,unharvestable_pods,4899.059312,4899.059312,0.000000,5049.266341,0.492451
1,2021-08-07 00:06:08,25,harvested_pods,0.000000,4899.059312,0.000000,5049.266341,0.492451
2,2021-08-07 00:06:08,25,harvestable_pods,5049.266341,4899.059312,0.000000,5049.266341,0.492451
3,2021-08-07 00:06:08,25,unharvestable_ratio,0.492451,4899.059312,0.000000,5049.266341,0.492451
4,2021-08-08 00:08:15,49,unharvestable_pods,18029.272057,18029.272057,15526.430505,337.098080,0.531950


In [10]:
colors = {
    'unharvestable_pods': '#ff595e', 
    'harvested_pods': "#8ac926", 
    'harvestable_pods': "#1982c4", 
    'unharvestable_ratio': "#6a4c93", 
} 
c = (
    chart(
        df, 
        "timestamp", 
        lmetrics=['unharvestable_pods', 'harvested_pods', 'harvestable_pods',], 
        rmetrics=['unharvestable_ratio'],
        lstrategy="stack_area", 
        rstrategy="line",
        title="Pod Line Breakdown", 
        tooltip_formats={
            'unharvestable_pods': ',d', 
            'harvested_pods': ',d', 
            'harvestable_pods': ',d', 
            'unharvestable_ratio': '.0%', 
        },
        yaxis_left_kwargs=dict(title="Pods", format=".3~s"), 
        yaxis_right_kwargs=dict(title="Ratio", format=".0%"), 
        dual_axes=True, 
        colors=colors, 
        show_exploit_rule=True, 
        exploit_day=16, 
    )
)

css_lines = css_tooltip_timeseries_multi_colored(value_vars, colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

c

alt.LayerChart(...)

In [11]:
# TODO: update css for this chart 
output_chart(c, css=css)

<IPython.core.display.JSON object>